# How to debug code

- corner cases: n=1 or maximum possible number
- think what the answer should be with pen and paper
- to measure CPU time since the start of the program, use "time.clock()"
- measure time for small, medium, and large tests

## Fabonacci numbers

- $F_{n} = 0$ if $n=0$
- $F_{n} = 1$ if $n=1$
- $F_{n} = F_{n-1} + F_{n-2}$ if $n>0$

### Naive algorithm

```
FibRecurse(n)
    if n <= 1
        return 1
    else
        FibRecurse(n-1) + FibRecurse(n-2)
```

### Efficient algorithm

```
FibList(n)
    create an array F[0...n]
    F[0] = 1
    F[1] = 1
    for i from 2 to n
        F[i] = F[i-1] + F[i-2]
    return F[n]
```

## Greatest common divisors

- for integers $a,b$, find the largest integer $d$ that divides both $a,b$

### Naive algorithm

```
gcd(a,b)
    best = 0
    for d from 1 to a+b
        if d % a == 0 and d % b ==0
            best = d
    return best
```

### Efficient algorithm

- lemma: let $a^{'}$ be the remainder when $a$ is divided by $b$, then gcd$(a,b)$ = gcd$(a^{'},b)$ = gcd$(b, a^{'})$

```
EuclidGCD(a,b)
    if b == 0
        return a
    a' = the remainder when a is divided by b
    return EuclidGCD(b,a')
```

# Greedy algorithms

## Car refueling problem

```
MinRefills(x,n,L)
    numRefills = 0
    currentRefill = 0
    while currentRefill <= n:
        lastRefill = currentRefull
        while (currentRefill <= n and x[currentRefill+1] - x[lastRefill] <= L):
            currentRefill = currentRefill + 1
        if currentRefill == lastRefill:
            return IMPOSSIBLE
        if currentRefill <= n:
            numRefills = numRefills + 1
    return numRefills
```

- currentRefill changes from $0$ to $n+1$, one by one
- numRefill changes from $0$ to at most $n$, one by one
- thus runs in $O(n)$

## Grouping children problem

- naive algorithm
```
MinGroups(C)
    m = len(C)
    for each partition into groups: (C = G1 Union G2 Union ... Union Gk)
        good = true
        for i from 1 to k:
            if max(Gi) - min(Gi) > 1:
                good = false
        if good:
            m = min(m, k)
    return m
```